In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, DICAlignedGrid
from bmcs_shear.dic_crack.dic_crack_list import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
from bmcs_shear.dic_crack import sz_tests_series_2023

In [ ]:
from cgitb import small


def validate_crack_tips(X_Ca, h_x):
    # Calculate the distances between neighboring points
    x_C, y_C = X_Ca.T
    dists = np.diff(x_C)

    # 
    if not np.all(dists > 0):
        raise ValueError("The horizontal positions of cracks" + 
                         "must be monotonically increasing.")

    return X_Ca

In [ ]:
test = sz_tests_series_2023.B10_TV1
dcl = sz_tests_series_2023.new_dcl(test)

In [ ]:
dcl.dsf.dic_grid.t = 0.8
X_tip_t_Ca = dcl.X_tip_t_Ca

In [ ]:
x_range, y_range = X_tip_t_Ca.T
x_0, y_0, x_1, y_1 = dcl.dsf.dic_grid.X_frame
y_range

In [ ]:
x_0_range = np.hstack([[x_0], x_range, [x_1]])
y_0_range = np.hstack([y_range[:1], y_range, y_range[-1:]])
x_1_range = np.copy(x_0_range)
y_1_range = np.ones_like(x_1_range) * y_1
y_0_range

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,3))
ax.scatter(x_0_range, y_0_range)
ax.scatter(x_1_range, y_1_range)
ax.set_aspect('equal')

In [ ]:
import ibvpy.api as ib

In [ ]:
n_E, n_F = x_0_range.size, 1
xmodel = ib.XDomainFEGrid(coord_min=(x_0, x_1), coord_max=(y_0, y_1),
                 integ_factor=1, shape=(n_E, n_F),  # number of elements!
                 fets=ib.FETS2D4Q());

In [ ]:
xmodel.mesh.n_grid_elems